In [22]:

import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [135]:

class Fitness:
    def __init__(self, route, data):
        self.route = route
        self.data  = data
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        pathDistance = 0
        for i in range(len(self.route)):
            if i == 0 or i == (len(self.route)-1):
                continue
            else:
                j = route[i-1]
                k = route[i]
                print(j)
                print(k)
                pathDistance += self.data[j][k]
        self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [136]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route


In [137]:

def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))

    return population



In [138]:
def rankRoutes(population, data):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i], data).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [142]:
def selection(popRanked, eliteSize):
    selectionResults = []
    no = 0
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
#     if no == 0:
#         print(df)
#         print(df["cum_sum"])
#         print(df["cum_perc"])
#         no += 1
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [143]:


def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool



In [144]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [145]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        
        
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [146]:


def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual



In [147]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [148]:
def nextGeneration(currentGen, eliteSize, mutationRate, data):
    popRanked = rankRoutes(currentGen, data)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [149]:


def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations, data):
    pop = initialPopulation(popSize, population)
    print("Initial distance: " + str(1 / rankRoutes(pop, data)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate, data)
    
    print("Final distance: " + str(1 / rankRoutes(pop, data)[0][1]))
    bestRouteIndex = rankRoutes(pop, data)[0][0]
    
    bestRoute = pop[bestRouteIndex]
    return bestRoute



In [150]:
with open("input.txt") as f:
    numbers = "0123456789"
    finalList = []
    lines = f.readlines()
    for i in range(len(lines)):
        lst = []
        corp = lines[i].split(" ")
        for j in corp:
            if len(j)> 0 and j[0] in numbers:
                lst.append(int(j))
        finalList.append(lst)  
#     print(finalList)
cityList = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
data = finalList
data

[[0, 66, 21, 300, 500, 26, 77, 69, 125, 650],
 [66, 0, 35, 115, 36, 65, 85, 90, 44, 54],
 [21, 35, 0, 450, 448, 846, 910, 47, 11, 145],
 [300, 115, 450, 0, 65, 478, 432, 214, 356, 251],
 [500, 36, 448, 65, 0, 258, 143, 325, 125, 39],
 [26, 65, 846, 478, 258, 0, 369, 256, 345, 110],
 [77, 85, 910, 432, 143, 369, 0, 45, 120, 289],
 [69, 90, 47, 214, 325, 256, 45, 0, 325, 981],
 [125, 44, 11, 356, 125, 345, 120, 325, 0, 326],
 [650, 54, 145, 251, 39, 110, 289, 981, 326, 0]]

In [151]:
final = geneticAlgorithm(population=cityList, popSize=100, eliteSize=10, mutationRate=0.01, generations=500, data=data)
distance_total = 0
for i in range(len(cityList)):
    if i == 0 or i == (len(cityList)-1):
        continue
    else:

        j = final[i-1]
        k = final[i]
        distance_total += data[j][k]
print(final)
print(distance_total)


7
10


IndexError: list index out of range

In [23]:


def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
    pop = initialPopulation(popSize, population)
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

